In [9]:
#imports of libraries for webscraping understat
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [65]:
# check match's soup
url_match = 'https://understat.com/match/5186'
response = requests.get(url_match)
soup_match = BeautifulSoup(response.text, 'html.parser')
soup_match

In [ ]:
#to check
#soup_match.find('div', class_='progress-bar').parent

In [73]:
#define function to get one match's stat
def get_match_stats(match_id):
    #get soup for match match_id
    url_match = 'https://understat.com/match/' + match_id
    response = requests.get(url_match)
    soup_match = BeautifulSoup(response.text, 'html.parser')

    #Add date to dictionary first
    breadcrumb = soup_match.find('ul', class_='breadcrumb')
    # Locate the date, which is the last <li> element in the breadcrumb
    date_li = breadcrumb.find_all('li')[-1]
    match_date = date_li.text.strip()
    # Add the date to the match stats dictionary
    match_stats = {'Date': match_date}

    # Extract team names and add to match_stats dict
    teams_section = soup_match.find('div', class_='progress-bar teams-titles')
    home_team = teams_section.find('div', class_='progress-home').text.strip()
    away_team = teams_section.find('div', class_='progress-away').text.strip()
    match_stats['Home Team'] = home_team 
    match_stats['Away Team'] = away_team
    
    #list of required stats and loop through them
    stats_titles = ['xG', 'DEEP', 'PPDA', 'xPTS']
    for stat_title in stats_titles:
        # Find the progress bar whose progress-title matches the stat_title
        stat_section = soup.find('div', class_='progress-title', string=stat_title)
        if stat_section:
            # Navigate to the parent progress-bar div
            parent = stat_section.find_parent('div', class_='progress-bar')
                
            # Find all progress-value divs within this section
            progress_values = parent.find_all('div', class_='progress-value')
                
            # Extract the text from the first and second progress-value divs
            home_value = progress_values[0].text.strip().replace('\n', '')
            away_value = progress_values[1].text.strip().replace('\n', '') if len(progress_values) > 1 else None
            match_stats[stat_title] = {'Home': home_value, 'Away': away_value}

    return match_stats

get_match_stats('5451')

{'Date': 'Aug 23 2014',
 'Home Team': 'Eintracht Frankfurt',
 'Away Team': 'Freiburg',
 'xG': {'Home': '0.91', 'Away': '2.06'},
 'DEEP': {'Home': '4', 'Away': '4'},
 'PPDA': {'Home': '20.64', 'Away': '15.29'},
 'xPTS': {'Home': '0.54', 'Away': '2.27'}}

In [77]:
url_league = 'https://understat.com/league/Bundesliga/2014'
response_league = requests.get(url_league)
soup_league = BeautifulSoup(response_league.text, 'html.parser')

In [140]:
year_list = range(2014,2025)
match_ids = []
for year in year_list:
    url_league = 'https://understat.com/league/Bundesliga/' + str(year)
    response_league = requests.get(url_league)
    soup_league = BeautifulSoup(response_league.text, 'html.parser')
    
    # Find the script tag containing the 'datesData' variable
    script = soup_league.find('script', text=lambda t: t and 'datesData' in t).string
    
    # Extract the JSON part from the script
    start_index = script.find("JSON.parse('") + len("JSON.parse('")
    end_index = script.find("')", start_index)
    json_data = script[start_index:end_index].encode('utf-8').decode('unicode_escape')
    
    # Parse the JSON data
    matches_data = json.loads(json_data)
    
    # Extract and add all match IDs to match id list
    for match in matches_data:
        match_ids.append(match['id'])
    
# Print the list of match IDs to check
print(match_ids)

{'27993', '28028', '27827', '27788', '27883', '28012', '27755', '27952', '27795', '27821', '27973', '28006', '28019', '27942', '27798', '27879', '27920', '27986', '28000', '28039', '27834', '27966', '27848', '27784', '27745', '27891', '27888', '27922', '27850', '27997', '27757', '28046', '27943', '27925', '28016', '27823', '27801', '27759', '27955', '27916', '27826', '27777', '27867', '27860', '27794', '27874', '27917', '27774', '27929', '27979', '28029', '27844', '27909', '28037', '27782', '27831', '27964', '27830', '27996', '27766', '27932', '27945', '27989', '27854', '27893', '28021', '27983', '28001', '27847', '28003', '27775', '27810', '27919', '27778', '27753', '27772', '27980', '28044', '27807', '27773', '27768', '27787', '28010', '28032', '28008', '27793', '27875', '27895', '27944', '27901', '27914', '27963', '27791', '27761', '27817', '27962', '28004', '27804', '27911', '28034', '27750', '27815', '27902', '27783', '27789', '27871', '27797', '27937', '28031', '27786', '27969', 

/var/folders/72/b7zxktp96cz1n4tjk3mlbj5w0000gn/T/ipykernel_987/4138678457.py:9: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script = soup_league.find('script', text=lambda t: t and 'datesData' in t).string


In [141]:
len(match_ids)
# -> 3366, also 11 saisons , weil /2024 die aktuelle saison ist und schon alle match_ids angelegt sind, 
#auch wenn die matches noch niht gespielt sind und die understat seite noch einen 404 fehler zeigt

306